# Preamble

In [1]:
using LinearAlgebra, ForwardDiff

In [2]:
Threads.nthreads()

10

In [14]:
lambda = 0.01;
xiG = 0;
xig = 4089;
alpha = (3e+4)^2/3;

n = 5;
xn = 4;

In [105]:
function metric(x)
    metric = zeros(eltype(x), n, n)
    
    xr2 = sum([x[i]^2 for i=1:xn])
    
    for i=1:xn, j=1:xn
        metric[i,j] = exp(-sqrt(2/3)*x[5])*(==(i,j) - 6*xiG^2*x[i]*x[j]/(1+xiG*xr2)) 
    end 
        
    metric[n,n] = 1
    
    return metric
end;

function viel(x)
    e,u = eigen(inv(metric(x)))
    return u*Diagonal([sqrt(ee) for ee in e])*inv(u)
end;

function gijk(x)
    metricdiff = ForwardDiff.jacobian(metric, x)
    gijk = [metricdiff[(i-1)*5+j,k] for i=1:n, j=1:n, k=1:n]
    return gijk
end;

function gijkl(x) 
    gijkl = zeros(eltype(x),n,n,n,n)
    
    for i=1:n, j=1:n
        gij(x) = metric(x)[i,j]
        gijhess = ForwardDiff.hessian(gij, x)
        
        for k=1:n, l=1:n
            gijkl[i,j,k,l] = gijhess[k,l]
        end;
    end;

    return gijkl
end;

function invgijk(x)
    invmetric = inv(metric(x))
    metricdiff = gijk(x)
    
    return [-invmetric[i,:]' * metricdiff[:,:,k] * invmetric[j,:] for i=1:n, j=1:n, k=1:n]
end;

function affine(x)
    invmetric = inv(metric(x))
    metricdiff = gijk(x)
    
    return [1/2*sum([invmetric[i,s]*(metricdiff[s,j,k] + metricdiff[s,k,j] - 
                    metricdiff[k,j,s]) for s=1:n]) for i=1:n, j=1:n, k=1:n]
end;

function Gammaijkl(x)
    invmetric = inv(metric(x))
    metricdiff = gijk(x)
    invgdiff = invgijk(x)
    ghess = gijkl(x)
    
    return [1/2*
        sum([invgdiff[i,s,l] * (metricdiff[s,j,k] + metricdiff[s,k,j] - metricdiff[k,j,s]) +
                invmetric[i,s]*(ghess[s,j,k,l] + ghess[s,k,j,l] - ghess[k,j,s,l]) 
                for s=1:n]) for i=1:n, j=1:n, k=1:n, l=1:n]
end;
    
function riemann(x)
    affinediff = Gammaijkl(x)
    Gamma = affine(x)
    
    return [affinediff[i,j,l,k] - affinediff[i,j,k,l] +
        sum([Gamma[s,j,l]*Gamma[i,k,s] - Gamma[s,j,k]*Gamma[i,l,s] for s=1:n]) 
        for i=1:n, j=1:n, k=1:n, l=1:n]
end;

function potU(x)
    xr2 = sum([x[i]^2 for i=1:xn])
    
    return lambda/4*exp(-2*sqrt(2/3)*x[5])*xr2^2 + 
        1/4/alpha*(1-exp(-sqrt(2/3)*x[5])*(1+(xig+xiG)*xr2))^2
end;

Uprime(x) = ForwardDiff.gradient(potU,x);

function Upp(x)
    invmetric = inv(metric(x))
    Gamma = affine(x)
    Up = Uprime(x)
    Uhess = ForwardDiff.hessian(potU,x)
    
    return invmetric * (Uhess - [sum([Gamma[k,i,j]*Up[k] for k=1:n]) for i=1:n, j=1:n])
end;

In [106]:
xs = [1.,2,3,4,5]
Upp(xs)

5×5 Matrix{Float64}:
  0.0382661     0.00443449   0.00665174   0.00886899  -0.0407281
  0.00443449    0.0449178    0.0133035    0.017738    -0.0814563
  0.00665174    0.0133035    0.0560041    0.026607    -0.122184
  0.00886899    0.017738     0.026607     0.0715248   -0.162913
 -0.000686902  -0.0013738   -0.00206071  -0.00274761   0.0112164